In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso, LassoCV, lasso_path
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from toyota.utils import load_dataset, get_metrics, LinearRegDiagnostic

toyota = cut_outliers.copy()

def lasso_selection(X_train, X_test, y_train, y_test, min_alpha=0, max_alpha=4, steps=100):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    alphas = np.logspace(min_alpha, max_alpha, steps)
    coefs = []
    rmse_list = []
    r2_list = []

    for a in alphas:
        clf = Lasso(alpha=a, max_iter=10000)
        clf.fit(X_train_scaled, y_train)
        coefs.append(clf.coef_)
        y_pred = clf.predict(X_test_scaled)
        mse = mean_squared_error(y_test, y_pred)
        rmse_list.append(np.sqrt(mse))
        r2 = r2_score(y_test, y_pred)
        r2_list.append(r2)

    alphas_idx = pd.Index(alphas, name="alpha")
    coefs_df = pd.DataFrame(coefs, index=alphas_idx, columns=X_train.columns)
    coefs_plot(coefs_df)
    rmse_plot(alphas, rmse_list)
    r2_plot(alphas, r2_list)
    ordered_coefs = print_coefs(coefs_df, rmse_list)
    worst_coefs = ordered_coefs[ordered_coefs == 0].index.tolist()
    return worst_coefs

def coefs_plot(coefs):
    sns.lineplot(data=coefs)
    plt.xscale("log")
    plt.xlabel("alpha")
    plt.ylabel("Coeficiente")
    plt.title("Trayectoria de coeficientes Lasso")
    plt.show()

def rmse_plot(alphas, rmse_list):
    sns.lineplot(x=alphas, y=rmse_list)
    plt.xscale("log")
    plt.ylabel("RMSE")
    min_rmse_idx = np.argmin(rmse_list)
    min_alpha = alphas[min_rmse_idx]
    min_rmse = rmse_list[min_rmse_idx]
    plt.plot(min_alpha, min_rmse, 'ro', label=f'Min RMSE: {min_rmse:.2f}\nAlpha: {min_alpha:.2e}')
    plt.legend()
    plt.title("RMSE vs alpha")
    plt.show()

def r2_plot(alphas, r2_list):
    sns.lineplot(x=alphas, y=r2_list)
    plt.xscale("log")
    plt.ylabel("R2")
    max_r2_idx = np.argmax(r2_list)
    max_alpha = alphas[max_r2_idx]
    max_r2 = r2_list[max_r2_idx]
    plt.plot(max_alpha, max_r2, 'ro', label=f'Max R2: {max_r2:.2f}\nAlpha: {max_alpha:.2e}')
    plt.legend()
    plt.title("R2 vs alpha")
    plt.show()

def print_coefs(coefs, mse_list):
    min_mse_idx = np.argmin(mse_list)
    coefs_df = np.abs(coefs.iloc[min_mse_idx]).sort_values(ascending=False)
    print(coefs_df)
    return coefs_df

Vamos a hacer la primera ejecucion

In [ ]:
X = toyota.drop(columns=["Price"], axis=1)
y = toyota["Price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
lasso_selection(X_train, X_test, y_train, y_test, min_alpha=-10, max_alpha=10, steps=200)

Se puede ajustar el minimo del alpha para reducir variables sin perder demasiada calidad en el modelo.

In [ ]:
lasso_selection(X_train, X_test, y_train, y_test, min_alpha=0.6, max_alpha=4, steps=200)

El modelo cuenta con muchas variables y parece empezar a empeorar rapidamente si seguimos aumentando el alpha minimo.

Hasta aqui hemos determinado el mejor modelo con: 

Features: 37:

lista de features: [
    "Mfg_Year",
    "KM",
    "Weight",
    "m_vvtli",
    "Automatic_airco",
    "Quarterly_Tax",
    "m_d4d",
    "CNG",
    "Guarantee_Period",
    "Diesel",
    "m_liftb",
    "HP",
    "Automatic",
    "m_hatch_b",
    "m_g6",
    "Powered_Windows",
    "m_sedan",
    "Mfr_Guarantee",
    "CD_Player",
    "Five_Doors",
    "m_comfort",
    "Airco",
    "Boardcomputer",
    "BOVAG_Guarantee",
    "Trunk",
    "Backseat_Divider",
    "Sport_Model",
    "Tow_Bar",
    "ABS",
    "Airbag_2",
    "Met_Color",
    "m_matic4",
    "Power_Steering",
    "Radio",
    "m_luna",
    "m_terra",
    "m_16v"
]

r2 = 0.91
rmse = 909.04
alpha = 3.98

### Alternativas con limpieza de outliers

Ahora, veamos alternativas de modelos que podrían resultar viables, pero con limpieza de outliers. Esta vez, asignaremos un margen medido en "desviaciones estandar" del error estandar y tomando las 3 mejores opciones de lambda maximo, ademas del lambda minimo, cuyos estadisticos del modelo se aproximen al mejor obtenido hasta ahora. Al final obtenemos la lista de indices de los outliers que pueden quitarse para mejorar el modelo de lasso. Usamos LassoCV.

Creamos la funcion que busca el rango de alpha

In [ ]:
def lasso_with_threshold(X, y, no_deviations=1):
    # Es decir, alpha entre exp(1) y exp(8)
    alphas_grid = np.logspace(-3, 6, 400)

    # Ajustar LassoCV con el rango de alphas adecuado
    lasso_cv = LassoCV(alphas=alphas_grid, cv=10, random_state=42)
    lasso_cv.fit(X, y)

    # Obtener alphas y errores
    alphas = lasso_cv.alphas_
    mse_path = lasso_cv.mse_path_
    mean_mse = np.mean(mse_path, axis=1)
    std_mse = np.std(mse_path, axis=1)

    # Índices y valores de alpha mínimo y 1-SE
    idx_min = np.argmin(mean_mse)
    alpha_min = alphas[idx_min]
    
    se_min = std_mse[idx_min]
    threshold = mean_mse[idx_min] + no_deviations * se_min  # 1 std deviation above the minimum MSE

    # Find the first alpha whose mean MSE is within 1 std deviation of the minimum
    idxs_se = np.where(mean_mse <= threshold)[0]
    if len(idxs_se) > 0:
        # Ensure alpha_se is the first such alpha (i.e., the largest/most regularized within the threshold)
        idx_se = idxs_se[0]
        idx_2_se = idxs_se[1]
        idx_3_se = idxs_se[2]
    else:
        # Fallback: use the minimum MSE index if none found (should not happen)
        idx_se = idx_min
        idx_2_se = idx_min
        idx_3_se = idx_min

    alpha_1 = alphas[idx_se]
    alpha_2 = alphas[idx_2_se]
    alpha_3 = alphas[idx_3_se]
    
    # Ruta de coeficientes para todas las alphas
    alphas_path, coefs_path, _ = lasso_path(X, y, alphas=alphas)
    n_features = (coefs_path != 0).sum(axis=0)
    # Crear gráfico con eje superior
    fig, ax = plt.subplots(figsize=(10, 6))

    # Curva de MSE con error estándar
    ax.errorbar(np.log(alphas), mean_mse, yerr=std_mse, fmt='-o', capsize=3, label='CV Error ± 1 SE')

    # Líneas verticales para alpha mínimo y 1-SE
    ax.axvline(np.log(alpha_min), color='r', linestyle='--', linewidth=2,
            label=f'min alpha ({n_features[idx_min]} features)')
    ax.axvline(np.log(alpha_1), color='g', linestyle=':', linewidth=2,
            label=f'1-SE alpha ({n_features[idx_se]} features)')
    ax.axvline(np.log(alpha_2), color='b', linestyle=':', linewidth=2,
            label=f'1-SE alpha ({n_features[idx_2_se]} features)')
    ax.axvline(np.log(alpha_3), color='y', linestyle=':', linewidth=2,
            label=f'1-SE alpha ({n_features[idx_3_se]} features)')
    
    # Etiquetas normales
    ax.set_xlabel('log(alpha)')
    ax.set_ylabel('MSE de validación cruzada')
    ax.set_title('Validación cruzada para Lasso')
    ax.legend()
    ax.grid(True)

    # Eje superior con número de features
    ax_top = ax.twiny()
    ax_top.set_xlim(ax.get_xlim())  # Sincronizar límites con eje inferior
    ax_top.set_xticks(np.log(alphas[::2]))  # Usar cada 2 alphas para no sobrecargar
    ax_top.set_xticklabels(n_features[::2])
    ax_top.set_xlabel('Número de features seleccionadas')

    plt.tight_layout()
    plt.show()
    
    # Obtener y mostrar los nombres de las features seleccionadas
    selected_min = X.columns[coefs_path[:, idx_min] != 0].tolist()
    selected_1 = X.columns[coefs_path[:, idx_se] != 0].tolist()
    selected_2 = X.columns[coefs_path[:, idx_2_se] != 0].tolist()
    selected_3 = X.columns[coefs_path[:, idx_3_se] != 0].tolist()
    
    
    # Imprimir de menor a mayor lambda..
    print(f"\nLambda minimo (base): {alpha_min} (log10={np.log10(alpha_min):.2f})")
    print(f"Features:")
    print(selected_min)
    
    print(f"\nTercera mejor opcion de lambra maximo: {alpha_3} (log10={np.log10(alpha_3):.2f})")
    print(f"Features:")
    print(selected_3)
    
    print(f"\nSegunda mejor opcion de lambra maximo: {alpha_2} (log10={np.log10(alpha_2):.2f})")
    print(f"Features:")
    print(selected_2)
    
    print(f"\nPrimera mejor opcion de lambda maximo: {alpha_1} (log10={np.log10(alpha_1):.2f})")
    print(f"Features:")
    print(selected_1)
    
    
    return [
        (alpha_min, selected_min),
        (alpha_3, selected_3),
        (alpha_2, selected_2),
        (alpha_1, selected_1),
    ]


Creamos la funcion simple de lasso

In [ ]:
X = toyota.drop(columns=["Price"], axis=1)
y = toyota["Price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [ ]:

def lasso_simple(X, y, features, alpha):

    X = X[features]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
    clf = Lasso()
    # Train the model with different regularisation strengths
    clf.set_params(alpha=alpha).fit(X_train, y_train)
    coef = clf.coef_
    y_pred = clf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print("Features: ", len(features))
    print("RMSE: ",rmse)
    print("R²: ",r2)
    return clf

Probamos 2 opciones de desviaciones estandar ya que son las mas proximas a los mejores modelos obtenidos anteriormente

Primero probamos con una tolerancia de 0.35 desviaciones estandar para el error

In [ ]:
res_list_1 = lasso_with_threshold(X_train, y_train, no_deviations=0.35)

In [ ]:
for res in res_list_1:
    lasso_simple(X, y, features=res[1], alpha=res[0])

Segundo probamos una tolerancia de 0.4 desviaciones para que pueda reducir mas el numero de features

In [ ]:
res_list_2 = lasso_with_threshold(X_train, y_train, no_deviations=0.4)

In [ ]:
for res in res_list_2:
    lasso_simple(X, y, features=res[1], alpha=res[0])

Ahora veamos si eliminando outliers podemos hacer algo con los mejores modelos obtenidos hasta el momento. Notamos que llega un punto en el que si seguimos sacando outliers el modelo comienza a empeorar

#### CASO 1 (Busqueda manual)
*Features:  37*

*RMSE:  909.04*

*R²:  0.91*

*Lambda = 3.98*

*Features: [ "Mfg_Year", "KM", "Weight", "m_vvtli", "Automatic_airco", "Quarterly_Tax", "m_d4d", "CNG", "Guarantee_Period", "Diesel", "m_liftb",    "HP",    "Automatic",    "m_hatch_b",    "m_g6",    "Powered_Windows",    "m_sedan", "Mfr_Guarantee",    "CD_Player",    "Five_Doors",    "m_comfort",    "Airco",    "Boardcomputer",    "BOVAG_Guarantee",    "Trunk", "Backseat_Divider",    "Sport_Model",    "Tow_Bar",    "ABS",    "Airbag_2",    "Met_Color",    "m_matic4",    "Power_Steering",
"Radio",    "m_luna",    "m_terra",    "m_16v"]*

#### CASO 2 (Busqueda con threshold)
*Features:  36*

*RMSE:  925.6536556988251*

*R²:  0.902661510112417*

*Lambda = 3.66*

*Features: ['Mfg_Year', 'KM', 'HP', 'Met_Color', 'Automatic', 'cc', 'Gears', 'Quarterly_Tax', 'Weight', 'Mfr_Guarantee', 'BOVAG_Guarantee', 'Guarantee_Period', 'ABS', 'Airbag_1', 'Airbag_2', 'Airco', 'Automatic_airco', 'Boardcomputer', 'CD_Player', 'Powered_Windows', 'Power_Steering', 'Metallic_Rim', 'Tow_Bar', 'CNG', 'm_16v', 'm_terra', 'm_liftb', 'm_wagon', 'm_sol', 'm_sedan', 'm_comfort', 'm_g6', 'm_d4d', 'm_gtsi', 'Trunk', 'Five_Doors']*



Creamos la funcion que nos determina la lista de outliers basado en la performance del Modelo de Lasso.

In [ ]:
# Identificador de outliers
def lasso_clean_outliers(selected_columns, alpha, outliers):
    print("Antes: ", toyota.shape)
    df_2 = toyota.drop(outliers, axis=0)
    print("Despues: ", df_2.shape)
    
    X = df_2.drop(columns=["Price"], axis=1)
    y = df_2["Price"]
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
    model = lasso_simple(X, y, features=selected_columns, alpha=alpha)

    from toyota.utils_lasso import SklearnLassoDiagnostic
    diagnosticPlotter = SklearnLassoDiagnostic(model, X[selected_columns], y, selected_columns)
    diagnosticPlotter()

    # Convert each outlier list (list of numpy arrays) to a DataFrame
    outlier_dfs = [
        pd.DataFrame(diagnosticPlotter.residuals_vs_fitted_outliers),
        pd.DataFrame(diagnosticPlotter.qq_plot_outliers),
        pd.DataFrame(diagnosticPlotter.scale_location_outliers),
        pd.DataFrame(diagnosticPlotter.leverage_plot_outliers)
    ]

    # Armamos una lista con todos los valores únicos de outlier_dfs
    all_outlier_values = []
    for df in outlier_dfs:
        all_outlier_values.extend(df.values.flatten())
    # Eliminamos duplicados y NaNs, y convertimos a enteros si corresponde
    all_outlier_values = [int(x) for x in set(all_outlier_values) if not pd.isnull(x)]

    # Para cada idx en all_outlier_values, buscamos el registro en X y luego su índice en toyota (sin "Price")
    toyota_no_price = toyota.drop(columns=["Price"], errors="ignore")

    global_indexes = []
    for idx in all_outlier_values:
        row = X[selected_columns].iloc[idx]
        # Buscamos la(s) fila(s) en toyota_no_price que coincidan exactamente con row
        mask = (toyota_no_price[selected_columns] == row.values).all(axis=1)
        # matching_indexes contiene los índices de toyota_no_price donde hay coincidencia exacta
        matching_indexes = toyota_no_price.index[mask].tolist()
        global_indexes.extend(matching_indexes)
        
    # Eliminamos duplicados
    global_indexes = list(set(global_indexes))
    print("Índices globales correspondientes a los outliers:", global_indexes)

Ahora veamos si mejora con la limpieza...

##### Caso 1

In [ ]:
features = ["Mfg_Year", "KM", "Weight", "m_vvtli", "Automatic_airco", "Quarterly_Tax", "m_d4d", "CNG", "Guarantee_Period", "Diesel", "m_liftb",    "HP",    "Automatic",    "m_hatch_b",    "m_g6",    "Powered_Windows",    "m_sedan", "Mfr_Guarantee",    "CD_Player",    "Five_Doors",    "m_comfort",    "Airco",    "Boardcomputer",    "BOVAG_Guarantee",    "Trunk", "Backseat_Divider",    "Sport_Model",    "Tow_Bar",    "ABS",    "Airbag_2",    "Met_Color",    "m_matic4",    "Power_Steering",
"Radio",    "m_luna",    "m_terra",    "m_16v"]
outliers = [1261, 14, 16, 49, 1432]
lasso_clean_outliers(features, alpha=3.98, outliers=outliers)

Vemos que empeora eliminando los primeros outliers encontrados.

##### Caso 2

In [ ]:
features = ['Mfg_Year', 'KM', 'HP', 'Met_Color', 'Automatic', 'cc', 'Gears', 'Quarterly_Tax', 'Weight', 'Mfr_Guarantee', 'BOVAG_Guarantee', 'Guarantee_Period', 'ABS', 'Airbag_1', 'Airbag_2', 'Airco', 'Automatic_airco', 'Boardcomputer', 'CD_Player', 'Powered_Windows', 'Power_Steering', 'Metallic_Rim', 'Tow_Bar', 'CNG', 'm_16v', 'm_terra', 'm_liftb', 'm_wagon', 'm_sol', 'm_sedan', 'm_comfort', 'm_g6', 'm_d4d', 'm_gtsi', 'Trunk', 'Five_Doors']
outliers = [
    16, 49, 14, 15, 330, 1133, 1261, 17, 1432, 
    410, 320, 1378, 388, 1131, 94, 991, 609,
    5, 102, 1072, 1268, 314, 1313, 163, 1353, 
    106, 19, 91, 64, 4, 114, 826, 668,
    # Si seguimos limpiando, Lasso no mejora.
]
lasso_clean_outliers(features, alpha=3.66, outliers=outliers)

Este nos permite limpiar el dataset y mejorar sustancialmente los estadisticos.

*Conclusion*: Hemos intentado mejorar 2 de los mejores modelos que teniamos hasta el momento con lasso, uno de 36 features y otro de 37 features. Pero al momento de borrar outliers, el modelo que mejoraba mas con la eliminacion de outliers es el de 36 variables.